# Latent Manipulation & Generation
**Purpose:** Implementation of the proposed **Latent Prompt Injection** method and generation of primary experimental data.

**Scope:**
* **Methodology:** Defines the core mathematical operator for injecting semantic prompts into the latent noise space ($\epsilon$-space).
* **Execution:** Runs the controlled generation loop using the proposed method.
* **Output:** Saves raw image tensors and metadata for downstream evaluation.

**Exclusions:**
* NO Baselines (See Notebook 03).
* NO Evaluation Metrics (See Notebook 04).
* NO Visualization or Plots (See Notebook 04).

## Execution Contract
1.  **Prerequisites:** `01_infrastructure_and_models.ipynb` must be executed first (or models/config available in memory/disk).
2.  **Determinism:** Uses the same locked global seeds defined in N1.
3.  **Artifacts:** All outputs are serialized to `outputs/proposed_method/`. No inline displays.

In [ ]:
import torch
import numpy as np
import os
from pathlib import Path
from tqdm.auto import tqdm

# Import shared utilities (assuming /src structure or previous notebook context) from src.config import config & src.utils import save_image_tensor
# If running as standalone notebook without /src package, ensure Config is loaded:

if 'config' not in globals():
    raise RuntimeError("Please run Notebook 01 first or load the configuration object.")
print("Imports complete. Ready for Latent Injection.")

In [ ]:
# Load Initialization Artifacts
import json

snapshot_path = config.OUTPUT_ROOT / "metadata" / "init_snapshot.json"
with open(snapshot_path, 'r') as f:
    snapshot = json.load(f)

# Assertions to ensure we are running on the intended infrastructure
assert snapshot['global_seed'] == config.Seed, "Seed Mismatch! Check N1."
assert snapshot['model_id'] == config.MODEL_ID, "Model ID Mismatch!"
assert str(config.DEVICE) == snapshot['device'], "Device Mismatch!"

print(f" Context Loaded. Reproducing run ID: {snapshot['timestamp']}")

In [ ]:
# Prompt & Conditioning Setup
PROMPTS = [
    "A cyberpunk city with neon lights",
    "A biological cell structure under microscope",
    "An astronaut riding a horse on mars",
    "A complex geometric fractal pattern"
]

NEGATIVE_PROMPT = "blurry, low quality, distortion, text, watermark"

def get_text_embeddings(prompt_list):
    text_input = tokenizer(
        prompt_list, 
        padding="max_length", 
        max_length=tokenizer.model_max_length, 
        truncation=True, 
        return_tensors="pt"
    )
    with torch.no_grad():
        embeddings = text_encoder(text_input.input_ids.to(config.DEVICE))[0]
    return embeddings
print(f"Loaded {len(PROMPTS)} standardized prompts for injection testing.")

## 3.1 Latent Space Injection Logic
**Formulation:**
Instead of relying solely on the text embedding $c$, we introduce a perturbation $\delta$ directly into the latent noise prediction process.

$$\hat{\epsilon} = \epsilon_\theta(z_t, t, c) + \lambda \cdot \mathcal{F}(z_{inject})$$

Where:
* $\epsilon_\theta$ is the frozen UNet.
* $\lambda$ is the injection strength.
* $\mathcal{F}$ is the proposed structural noise mapping.

In [ ]:
# PROPOSED LATENT INJECTION OPERATOR
class LatentInjector:
    def __init__(self, strength=0.5, injection_steps=20):
        self.strength = strength
        self.injection_steps = injection_steps
    
    def generate_noise_pattern(self, shape, pattern_type="fractal"):
        """
        Generates the structured noise map \delta described in Eq. 3.
        """
        noise = torch.randn(shape, device=config.DEVICE, dtype=config.PRECISION)
        if pattern_type == "fractal":
            pass 
        return noise

    def apply(self, latents, timestep, noise_pattern):
        """
        Applies the perturbation to the latents.
        """
        if timestep > self.injection_steps:
            return latents + (noise_pattern * self.strength)
        return latents

injector = LatentInjector(strength=0.8, injection_steps=15)
print("Latent Injection Operator initialized.")

In [ ]:
# Controlled Generation Loop
save_dir = config.OUTPUT_ROOT / "outputs" / "proposed_method"
save_dir.mkdir(parents=True, exist_ok=True)
print(f"Starting Generation. Output: {save_dir}")

cond_embeddings = get_text_embeddings(PROMPTS)
uncond_embeddings = get_text_embeddings([""] * len(PROMPTS))
text_embeddings = torch.cat([uncond_embeddings, cond_embeddings])
generated_images = []

for i, prompt in enumerate(PROMPTS):
    generator_seed = torch.Generator(device=config.DEVICE).manual_seed(config.Seed + i)
    latents = torch.randn(
        (1, unet.config.in_channels, config.LATENT_RES, config.LATENT_RES),
        generator=generator_seed,
        device=config.DEVICE,
        dtype=config.PRECISION
    )
    
    pattern = injector.generate_noise_pattern(latents.shape)
    latents = injector.apply(latents, timestep=999, noise_pattern=pattern)
    
    scheduler.set_timesteps(50)
    for t in scheduler.timesteps:
        latent_model_input = torch.cat([latents] * 2)
        latent_model_input = scheduler.scale_model_input(latent_model_input, t)
        
        with torch.no_grad():
            noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings[i*2:(i+1)*2]).sample
        noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
        noise_pred = noise_pred_uncond + 7.5 * (noise_pred_text - noise_pred_uncond)
        
        latents = scheduler.step(noise_pred, t, latents).prev_sample
    with torch.no_grad():
        image = vae.decode(latents / vae.config.scaling_factor).sample
    generated_images.append(image.cpu())
print(f"Generation Complete. {len(generated_images)} samples produced.")

In [ ]:
# Output Serialization
import torchvision.transforms as T
from PIL import Image

to_pil = T.ToPILImage()
metadata_log = []
for idx, (img_tensor, prompt) in enumerate(zip(generated_images, PROMPTS)):
    img_tensor = (img_tensor / 2 + 0.5).clamp(0, 1).squeeze(0)
    file_name = f"proposed_sample_{idx:03d}.png"
    pil_img = to_pil(img_tensor.float())
    pil_img.save(save_dir / file_name)

    metadata_log.append({
        "file_name": file_name,
        "prompt": prompt,
        "seed": config.Seed + idx,
        "method": "latent_injection",
        "injection_strength": injector.strength
    })

with open(save_dir / "generation_log.json", "w") as f:
    json.dump(metadata_log, f, indent=4)
print(f"Serialized {len(metadata_log)} assets to {save_dir}")

In [ ]:
# Sanity Checks
expected_count = len(PROMPTS)
actual_count = len(list(save_dir.glob("*.png")))
assert expected_count == actual_count, f"Missing outputs! Expected {expected_count}, found {actual_count}"
for img in generated_images:
    assert not torch.isnan(img).any(), "FATAL: Generated image contains NaNs."
print("Sanity Checks Passed. Artifacts are valid.")

## Phase 2 Complete
**Artifacts Produced:**
1.  `outputs/proposed_method/*.png` - Raw images generated by Latent Injection.
2.  `outputs/proposed_method/generation_log.json` - Metadata for the Evaluation Notebook.

**Next Steps:**
Proceed to **Notebook 03: `Baselines & Ablations.ipynb`** to generate the comparative data (ControlNet, P2P) required for the manuscript tables.